In [68]:
!pip install python-bcb pandas seaborn sklearn yfinance

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-

In [69]:
import pandas as pd
import seaborn as sns
import yfinance as yf
from bcb import sgs
from sklearn.feature_selection import mutual_info_classif, chi2
from sklearn.preprocessing import MinMaxScaler

In [70]:
tickers = ['^GSPC', '^VIX', '^TNX',  'BZ=F', 'BRL=X', 'IRFM11.SA']
dataframes = []
closes = []
for ticker in tickers:
    dado = yf.download(
        ticker,
        start='2020-01-01',
        end='2025-08-01',
        auto_adjust=True
    )

    if not dado.empty:
        close = dado[['Close']].rename(columns={'Close': ticker})
        closes.append(close)
df_final = pd.concat(closes, axis=1)
print(df_final)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Price             ^GSPC       ^VIX   ^TNX       BZ=F     BRL=X  IRFM11.SA
Ticker            ^GSPC       ^VIX   ^TNX       BZ=F     BRL=X  IRFM11.SA
Date                                                                     
2020-01-01          NaN        NaN    NaN        NaN  4.016786        NaN
2020-01-02  3257.850098  12.470000  1.882  66.250000  4.016300  61.880001
2020-01-03  3234.850098  14.020000  1.788  68.599998  4.023400  61.959999
2020-01-06  3246.280029  13.850000  1.811  68.910004  4.057000  61.950001
2020-01-07  3237.179932  13.790000  1.827  68.269997  4.060400  61.939999
...                 ...        ...    ...        ...       ...        ...
2025-07-25  6388.640137  14.930000  4.386  68.440002  5.520000  90.989998
2025-07-28  6389.770020  15.030000  4.420  70.040001  5.565200  91.070000
2025-07-29  6370.859863  15.980000  4.330  72.510002  5.586800  91.279999
2025-07-30  6362.899902  15.480000  4.376  73.239998  5.573200  91.330002
2025-07-31  6339.390137  16.719999  4.

In [71]:
inicio = '2020-01-01'
fim = '2025-08-01'

ipca_exp = sgs.get(433, start=inicio, end=fim)
ipca_exp.rename(columns={433: 'IPCA_Expectativa_12m'}, inplace=True)

selic = sgs.get(432, start=inicio, end=fim)
selic.rename(columns={432: 'Selic_Meta'}, inplace=True)

ibcbr = sgs.get(24363, start=inicio, end=fim)
ibcbr.rename(columns={24363: 'IBC_Br'}, inplace=True)

ptax = sgs.get(1, start=inicio, end=fim)
ptax.rename(columns={1: 'Dolar_PTAX'}, inplace=True)

macro_bcb = {
    'IPCA_EXP': ipca_exp,
    'SELIC': selic,
    'IBC_BR': ibcbr,
    'PTAX': ptax
}
macro_bcb['SELIC'].describe()

macro_df = pd.concat(macro_bcb.values(), axis=1)
macro_df.head()
macro_df = macro_df.ffill()
macro_df.head()

,433,432,24363,1
Date,,,,
2020-01-01,0.21,4.5,94.02696,NaN
2020-01-02,0.21,4.5,94.02696,4.0213
2020-01-03,0.21,4.5,94.02696,4.0522
2020-01-04,0.21,4.5,94.02696,4.0522
2020-01-05,0.21,4.5,94.02696,4.0522


In [72]:
join = [macro_df, df_final]

externo = pd.concat(join, axis=1)
print(externo)

             433   432      24363       1  (^GSPC, ^GSPC)  (^VIX, ^VIX)  \
Date                                                                      
2020-01-01  0.21   4.5   94.02696     NaN             NaN           NaN   
2020-01-02  0.21   4.5   94.02696  4.0213     3257.850098     12.470000   
2020-01-03  0.21   4.5   94.02696  4.0522     3234.850098     14.020000   
2020-01-04  0.21   4.5   94.02696  4.0522             NaN           NaN   
2020-01-05  0.21   4.5   94.02696  4.0522             NaN           NaN   
...          ...   ...        ...     ...             ...           ...   
2025-07-28  0.26  15.0  113.16809  5.5877     6389.770020     15.030000   
2025-07-29  0.26  15.0  113.16809  5.5764     6370.859863     15.980000   
2025-07-30  0.26  15.0  113.16809  5.6034     6362.899902     15.480000   
2025-07-31  0.26  15.0  113.16809  5.6021     6339.390137     16.719999   
2025-08-01 -0.11  15.0  110.42518  5.5436             NaN           NaN   

            (^TNX, ^TNX)

In [73]:
url_focus = (
    "https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/odata/"
    "ExpectativasMercadoAnuais?$top=100000"
    "&$filter=Indicador%20eq%20'IPCA'%20and%20Data%20ge%20'2020-01-01'"
    "&$format=json"
    "&$orderby=Data%20asc"
)
try:
    json = pd.read_json(url_focus)
    Focus = pd.DataFrame(json['value'].tolist())

    Focus['Data'] = pd.to_datetime(Focus['Data'])

    Focus['DataReferencia'] = Focus['DataReferencia'].astype(int)
    
    Focus['Ano_Divulgacao'] = Focus['Data'].dt.year
    
    Focus = Focus[Focus['DataReferencia'] == Focus['Ano_Divulgacao']].copy()

    Focus = Focus.set_index('Data').sort_index()
    
    Focus = Focus[['Mediana']].rename(columns={'Mediana': 'IPCA_Expectativa_AnoCorrente'})
    
    print(Focus.tail())

except Exception as e:
    raise TypeError
print(Focus)

            IPCA_Expectativa_AnoCorrente
Data                                    
2025-12-10                        4.3579
2025-12-11                        4.3683
2025-12-11                        4.3579
2025-12-12                        4.3579
2025-12-12                        4.3540
            IPCA_Expectativa_AnoCorrente
Data                                    
2020-01-02                        3.6000
2020-01-02                        3.5800
2020-01-03                        3.6000
2020-01-03                        3.5900
2020-01-06                        3.6000
...                                  ...
2025-12-10                        4.3579
2025-12-11                        4.3683
2025-12-11                        4.3579
2025-12-12                        4.3579
2025-12-12                        4.3540

[2990 rows x 1 columns]


In [74]:
Focus = Focus.groupby(Focus.index).mean()
Focus = Focus.sort_index()
print(Focus)

            IPCA_Expectativa_AnoCorrente
Data                                    
2020-01-02                       3.59000
2020-01-03                       3.59500
2020-01-06                       3.58000
2020-01-07                       3.58000
2020-01-08                       3.58500
...                                  ...
2025-12-08                       4.39420
2025-12-09                       4.38960
2025-12-10                       4.36310
2025-12-11                       4.36310
2025-12-12                       4.35595

[1495 rows x 1 columns]


In [75]:
FeatureBase = pd.merge(externo, Focus, left_index=True, right_index=True, how='inner')
FeatureSemanal = FeatureBase[FeatureBase.index.dayofweek == 4].copy()
print(FeatureSemanal)

             433   432      24363       1  (^GSPC, ^GSPC)  (^VIX, ^VIX)  \
2020-01-03  0.21   4.5   94.02696  4.0522     3234.850098         14.02   
2020-01-10  0.21   4.5   94.02696  4.0745     3265.350098         12.56   
2020-01-17  0.21   4.5   94.02696  4.1837     3329.620117         12.10   
2020-01-24  0.21   4.5   94.02696  4.1769     3295.469971         14.56   
2020-01-31  0.21   4.5   94.02696  4.2695     3225.520020         18.84   
...          ...   ...        ...     ...             ...           ...   
2025-07-04  0.26  15.0  113.16809  5.4090             NaN           NaN   
2025-07-11  0.26  15.0  113.16809  5.5722     6259.750000         16.40   
2025-07-18  0.26  15.0  113.16809  5.5466     6296.790039         16.41   
2025-07-25  0.26  15.0  113.16809  5.5426     6388.640137         14.93   
2025-08-01 -0.11  15.0  110.42518  5.5436             NaN           NaN   

            (^TNX, ^TNX)  (BZ=F, BZ=F)  (BRL=X, BRL=X)  \
2020-01-03         1.788     68.599998   

In [76]:
finbert = pd.read_csv('FinBert.csv')
finbert['data'] = pd.to_datetime(finbert['data'])

Diario = finbert.groupby('data')['score'].sum().sort_index()
Diario = Diario.to_frame(name='scoreTotal')

Diario['scoreSemanal'] = Diario['scoreTotal'].rolling('7D', min_periods=1).mean()

FeatureB = Diario.loc[Diario.index.dayofweek == 4, ['scoreSemanal']].copy()

print(FeatureB.tail())

            scoreSemanal
data                    
2025-09-26      4.593504
2025-10-03      3.399397
2025-10-10      3.692898
2025-10-17      3.071258
2025-10-24      3.468857


In [77]:
Features_B = pd.merge(FeatureB, FeatureSemanal, left_index=True, right_index=True, how='inner')
print(Features_B)

            scoreSemanal   433   432      24363       1  (^GSPC, ^GSPC)  \
2020-01-03      0.724997  0.21   4.5   94.02696  4.0522     3234.850098   
2020-01-10      4.051150  0.21   4.5   94.02696  4.0745     3265.350098   
2020-01-17      3.963479  0.21   4.5   94.02696  4.1837     3329.620117   
2020-01-24      3.780082  0.21   4.5   94.02696  4.1769     3295.469971   
2020-01-31      3.870058  0.21   4.5   94.02696  4.2695     3225.520020   
...                  ...   ...   ...        ...     ...             ...   
2025-07-04      2.580007  0.26  15.0  113.16809  5.4090             NaN   
2025-07-11      2.015503  0.26  15.0  113.16809  5.5722     6259.750000   
2025-07-18      2.108633  0.26  15.0  113.16809  5.5466     6296.790039   
2025-07-25      1.776105  0.26  15.0  113.16809  5.5426     6388.640137   
2025-08-01      1.660700 -0.11  15.0  110.42518  5.5436             NaN   

            (^VIX, ^VIX)  (^TNX, ^TNX)  (BZ=F, BZ=F)  (BRL=X, BRL=X)  \
2020-01-03         14.02   

In [78]:
nans = Features_B.isna().sum()
print(nans[nans > 0])

(^GSPC, ^GSPC)             4
(^VIX, ^VIX)               4
(^TNX, ^TNX)               4
(BZ=F, BZ=F)               3
(BRL=X, BRL=X)             1
(IRFM11.SA, IRFM11.SA)    15
dtype: int64


In [79]:
Features_B = Features_B.ffill()
Features_B = Features_B.dropna()

print(f"Total de NaNs: {Features_B.isna().sum().sum()}")

Total de NaNs: 0


In [80]:
Features_B.columns

Features_B.columns = [
    'scoreSemanal',
    'ipcaMensal',
    'selicMeta',
    'ibcBrActivity',
    'dolarPtax',
    'sp500Index',
    'vixIndex',
    'treasuryYield10y',
    'brentOilPrice',
    'usdBrlExchange',
    'irfm11FixedIncome',
    'IPCA_Ano_Atual'
]

Features_B.columns

Index(['scoreSemanal', 'ipcaMensal', 'selicMeta', 'ibcBrActivity', 'dolarPtax',
       'sp500Index', 'vixIndex', 'treasuryYield10y', 'brentOilPrice',
       'usdBrlExchange', 'irfm11FixedIncome', 'IPCA_Ano_Atual'],
      dtype='object')

In [81]:
print(Features_B)

            scoreSemanal  ipcaMensal  selicMeta  ibcBrActivity  dolarPtax  \
2020-01-03      0.724997        0.21        4.5       94.02696     4.0522   
2020-01-10      4.051150        0.21        4.5       94.02696     4.0745   
2020-01-17      3.963479        0.21        4.5       94.02696     4.1837   
2020-01-24      3.780082        0.21        4.5       94.02696     4.1769   
2020-01-31      3.870058        0.21        4.5       94.02696     4.2695   
...                  ...         ...        ...            ...        ...   
2025-07-04      2.580007        0.26       15.0      113.16809     5.4090   
2025-07-11      2.015503        0.26       15.0      113.16809     5.5722   
2025-07-18      2.108633        0.26       15.0      113.16809     5.5466   
2025-07-25      1.776105        0.26       15.0      113.16809     5.5426   
2025-08-01      1.660700       -0.11       15.0      110.42518     5.5436   

             sp500Index  vixIndex  treasuryYield10y  brentOilPrice  \
2020-

Pre-processamento antes do XGBOOST

In [82]:
import numpy as np

ativos = pd.read_csv('ibovespa.csv', sep=';')
dataset = {}
tickers = ativos['ticker']

for ticker in tickers:
    df = yf.download(
        ticker,
        start='2020-01-01',
        end = '2025-08-01',
        auto_adjust=True,
        progress=False
    )
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)

    df['AlvoRetorno'] = df['Close'].shift(-5) / df['Close'] - 1
    df['Alvo'] = np.where(df['AlvoRetorno'] > 0, 1, 0)
    df = df[df.index.weekday == 4]

    df = df.drop(columns=['Open', 'High', 'Low', 'Close', 'Volume'], errors='ignore')

    var = pd.concat([df, Features_B], axis=1)
    var = var.dropna()
    var = var.dropna(subset=['Alvo', 'AlvoRetorno'])
    var['Alvo'] = var['Alvo'].astype(int)
    dataset[ticker] = var

print("\nProcessamento Finalizado.")
print(f"Temos {len(dataset)} ativos prontos para modelagem.")
    


Processamento Finalizado.
Temos 59 ativos prontos para modelagem.


In [83]:
print(dataset['ITUB4.SA'].columns)

Index(['AlvoRetorno', 'Alvo', 'scoreSemanal', 'ipcaMensal', 'selicMeta',
       'ibcBrActivity', 'dolarPtax', 'sp500Index', 'vixIndex',
       'treasuryYield10y', 'brentOilPrice', 'usdBrlExchange',
       'irfm11FixedIncome', 'IPCA_Ano_Atual'],
      dtype='object')


In [84]:
import numpy as np

for tick, df in dataset.items():
    featuresCol = [c for c in df.columns if c not in ['Alvo', 'AlvoRetorno','scoreSemanal']]
    
    x = df[featuresCol]
    
    corr_matrix = x.corr().abs()

    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

    drop = [column for column in upper.columns if any(upper[column] > 0.95)]

    if len(drop) > 0:
        print(f"[{tick}] Removendo {len(drop)} features redundantes: {drop}")
        
        dataset[tick] = df.drop(columns=drop)
print("\nTratamento de correlação concluido")

[PETR4.SA] Removendo 1 features redundantes: ['usdBrlExchange']
[ITUB4.SA] Removendo 1 features redundantes: ['usdBrlExchange']
[VALE3.SA] Removendo 1 features redundantes: ['usdBrlExchange']
[BPAC11.SA] Removendo 1 features redundantes: ['usdBrlExchange']
[ABEV3.SA] Removendo 1 features redundantes: ['usdBrlExchange']
[WEGE3.SA] Removendo 1 features redundantes: ['usdBrlExchange']
[BBDC3.SA] Removendo 1 features redundantes: ['usdBrlExchange']
[ITSA4.SA] Removendo 1 features redundantes: ['usdBrlExchange']
[BBAS3.SA] Removendo 1 features redundantes: ['usdBrlExchange']
[SANB11.SA] Removendo 1 features redundantes: ['usdBrlExchange']
[VIVT3.SA] Removendo 1 features redundantes: ['usdBrlExchange']
[SBSP3.SA] Removendo 1 features redundantes: ['usdBrlExchange']
[BBSE3.SA] Removendo 1 features redundantes: ['usdBrlExchange']
[B3SA3.SA] Removendo 1 features redundantes: ['usdBrlExchange']
[SUZB3.SA] Removendo 1 features redundantes: ['usdBrlExchange']
[CPFE3.SA] Removendo 1 features redund

In [85]:
print(dataset['ITUB4.SA'].columns)

Index(['AlvoRetorno', 'Alvo', 'scoreSemanal', 'ipcaMensal', 'selicMeta',
       'ibcBrActivity', 'dolarPtax', 'sp500Index', 'vixIndex',
       'treasuryYield10y', 'brentOilPrice', 'irfm11FixedIncome',
       'IPCA_Ano_Atual'],
      dtype='object')


In [90]:
!pip install xgboost optuna scikit-learn

  Using cached xgboost-3.1.2-py3-none-win_amd64.whl.metadata (2.1 kB)
  Using cached optuna-4.6.0-py3-none-any.whl.metadata (17 kB)
   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
    --------------------------------------- 1.3/72.0 MB 8.4 MB/s eta 0:00:09
   -- ------------------------------------- 3.7/72.0 MB 10.0 MB/s eta 0:00:07
   --- ------------------------------------ 6.3/72.0 MB 10.7 MB/s eta 0:00:07
   ---- ----------------------------------- 8.7/72.0 MB 10.9 MB/s eta 0:00:06
   ------ --------------------------------- 11.3/72.0 MB 11.0 MB/s eta 0:00:06
   ------- -------------------------------- 13.6/72.0 MB 11.1 MB/s eta 0:00:06
   --------- ------------------------------ 16.3/72.0 MB 11.2 MB/s eta 0:00:05
   ---------- ----------------------------- 18.6/72.0 MB 11.3 MB/s eta 0:00:05
   ----------- ---------------------------- 21.2/72.0 MB 11.3 MB/s eta 0:00:05
   ------------- -------------------------- 23.6/72.0 MB 11.3 MB/s eta 0:00:05
   ----------

In [ ]:
import xgboost as xgb
import optuna
from sklearn.metrics import accuracy_score, precision_score
from sklearn.model_selection import train_test_split
import warnings

optuna.logging.set_verbosity(optuna.logging.WARNING)
warnings.filterwarnings('ignore')

modelos = {}


for ticker, df in dataset.items():
    print(f"\n--- Treinando: {ticker} ---")
    
    features = [c for c in df.columns if c not in ['Alvo', 'AlvoRetorno']]
    X = df[features]
    y = df['Alvo']

    split = int(len(df) * 0.8)
    
    X_dev = X.iloc[:split]
    y_dev = y.iloc[:split]
    
    X_backtest = X.iloc[split:]
    y_backtest = y.iloc[split:]
    
    def objective(trial):

        cutoff = int(len(X_dev) * 0.75)
        X_train_opt, X_val_opt = X_dev.iloc[:cutoff], X_dev.iloc[cutoff:]
        y_train_opt, y_val_opt = y_dev.iloc[:cutoff], y_dev.iloc[cutoff:]
        
        params = {
            'objective': 'binary:logistic',
            'eval_metric': 'logloss',
            'n_estimators': trial.suggest_int('n_estimators', 50, 500),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),
            'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 10.0),
            'gamma': trial.suggest_float('gamma', 0.0, 5.0),
            'random_state': 42,
            'n_jobs': -1
        }
        
        model = xgb.XGBClassifier(**params)
        model.fit(X_train_opt, y_train_opt)
        
        preds = model.predict(X_val_opt)
        
        score = precision_score(y_val_opt, preds, zero_division=0)
        
        return score

    study = optuna.create_study(direction='maximize')
    
    study.optimize(objective, n_trials=100) 
    
    print(f"Melhor Score: {study.best_value:.2%}")
    print(f"Melhores Parâmetros: {study.best_params}")

    best_params = study.best_params
    best_params['random_state'] = 42
    best_params['n_jobs'] = -1
    
    final_model = xgb.XGBClassifier(**best_params)
    final_model.fit(X_dev, y_dev)
    
    modelos[ticker] = final_model


Iniciando Otimização e Treinamento com XGBoost...

--- Treinando: PETR4.SA ---
Melhor Score (Validação): 100.00%
Melhores Parâmetros: {'n_estimators': 142, 'max_depth': 3, 'learning_rate': 0.20802049729756927, 'subsample': 0.6448278085028865, 'colsample_bytree': 0.6589386506780799, 'reg_alpha': 3.8274282961846775, 'reg_lambda': 6.730796763006314, 'gamma': 0.6870068426748621}

--- Treinando: ITUB4.SA ---
Melhor Score (Validação): 100.00%
Melhores Parâmetros: {'n_estimators': 204, 'max_depth': 8, 'learning_rate': 0.22347205721980834, 'subsample': 0.8466916316291433, 'colsample_bytree': 0.7723662088222658, 'reg_alpha': 0.014659468188664704, 'reg_lambda': 2.0362273024298707, 'gamma': 1.7490454216864588}

--- Treinando: VALE3.SA ---
Melhor Score (Validação): 100.00%
Melhores Parâmetros: {'n_estimators': 444, 'max_depth': 6, 'learning_rate': 0.06758844090628004, 'subsample': 0.75088755819102, 'colsample_bytree': 0.6340134530966473, 'reg_alpha': 1.3243007739811519, 'reg_lambda': 9.79193561308